In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import pandas as pd
import numpy as np
import glob
from newstock.news import Newstock

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [4]:
import sklearn.metrics as m

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [6]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
# font_location = '/usr/share/fonts/truetype/nanum/NanumGothicOTF.ttf'
font_location = 'C:/Windows/Fonts/gulim.ttc' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, text='text'):
        self.text = text

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.text]

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class Buy_Sell_transformer_slope(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, cut_percent=0.2):
        self.cut_percent = 1-cut_percent

    def fit(self, y):
        return self

    def transform(self, y):
        y = y.apply(lambda x: x.coef_[0][0])
        cut_value = y.quantile(self.cut_percent)
        print(cut_value)
        y = pd.cut(y, bins=[-np.inf, cut_value, np.inf], labels=['nothing','buy'])
        return y

In [9]:
import pandas as pd
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

class TimeBasedCV(object):
    '''
    Parameters 
    ----------
    train_period: int
        number of time units to include in each train set
        default is 30
    test_period: int
        number of time units to include in each test set
        default is 7
    freq: string
        frequency of input parameters. possible values are: days, months, years, weeks, hours, minutes, seconds
        possible values designed to be used by dateutil.relativedelta class
        deafault is days
    '''
    
    
    def __init__(self, train_period=30, test_period=7, freq='days'):
        self.train_period = train_period
        self.test_period = test_period
        self.freq = freq

        
        
    def split(self, data, validation_split_date=None, date_column='record_date', gap=0):
        '''
        Generate indices to split data into training and test set
        
        Parameters 
        ----------
        data: pandas DataFrame
            your data, contain one column for the record date 
        validation_split_date: datetime.date()
            first date to perform the splitting on.
            if not provided will set to be the minimum date in the data after the first training set
        date_column: string, deafult='record_date'
            date of each record
        gap: int, default=0
            for cases the test set does not come right after the train set,
            *gap* days are left between train and test sets
        
        Returns 
        -------
        train_index ,test_index: 
            list of tuples (train index, test index) similar to sklearn model selection
        '''
        
        # check that date_column exist in the data:
        try:
            data[date_column]
        except:
            raise KeyError(date_column)
                    
        train_indices_list = []
        test_indices_list = []

        if validation_split_date==None:
            validation_split_date = data[date_column].min().date() + eval('relativedelta('+self.freq+'=self.train_period)')
        
        start_train = validation_split_date - eval('relativedelta('+self.freq+'=self.train_period)')
        end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
        start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
        end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        while end_test <= data[date_column].max().date() + eval('relativedelta('+self.freq+'=1)'):
            # train indices:
            cur_train_indices = list(data[(data[date_column].dt.date>=start_train) & 
                                     (data[date_column].dt.date<end_train)].index)

            # test indices:
            cur_test_indices = list(data[(data[date_column].dt.date>=start_test) &
                                    (data[date_column].dt.date<end_test)].index)
            
            print("Train period:",start_train,"<= day <" , end_train, ", Test period", start_test, "<= day <", end_test,
                  "# train records", len(cur_train_indices), ", # test records", len(cur_test_indices))

            train_indices_list.append(cur_train_indices)
            test_indices_list.append(cur_test_indices)

            # update dates:
            start_train = start_train + eval('relativedelta('+self.freq+'=self.test_period)')
            end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
            start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
            end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        # mimic sklearn output  
        index_output = [(train,test) for train,test in zip(train_indices_list,test_indices_list)]

        self.n_splits = len(index_output)
        
        return index_output
    
    
    def get_n_splits(self):
        """Returns the number of splitting iterations in the cross-validator
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        return self.n_splits 

In [10]:
names = os.listdir('./stock_data/stock_article0301_0531/article/')

In [11]:
data = {}
for name in names:
    print(name)
    ns = Newstock(name)
    data[name] = {'X' : ns.news_data_with_price}

KB금융
LG생활건강
LG화학
NAVER
POSCO
SK
SK텔레콤
SK하이닉스
기아차
삼성SDI
삼성물산
삼성바이오로직스
삼성에스디에스
삼성전자
셀트리온
신한지주
엔씨소프트
카카오
현대모비스
현대차


In [18]:
train_period=60
test_period=29
tscv = TimeBasedCV(train_period=train_period, test_period=test_period, freq='days')
cv_sets = tscv.split(data['LG생활건강']['X'].reset_index(drop=True), validation_split_date=None, date_column='timestamps', gap=0)

Train period: 2020-03-02 <= day < 2020-05-01 , Test period 2020-05-01 <= day < 2020-05-30 # train records 735 , # test records 272


In [125]:
tfidf = TfidfVectorizer(sublinear_tf=False)
tfidf.fit(data['LG생활건강']['X']['text'].iloc[cv_sets[0][0]])

tfidf_s = TfidfVectorizer(sublinear_tf=True)
tfidf_s.fit(data['LG생활건강']['X']['text'].iloc[cv_sets[0][0]])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [126]:
test_vec = tfidf.transform(data['LG생활건강']['X']['text'].iloc[cv_sets[0][1]])
test_vec_s = tfidf_s.transform(data['LG생활건강']['X']['text'].iloc[cv_sets[0][1]])

In [127]:
test_vec.shape

(272, 28301)

In [128]:
len(data['LG생활건강']['X']['text'].iloc[cv_sets[0][1]])

272

In [129]:
test_vec[0, :].toarray().argsort()[0][::-1][:10]

array([ 5044,  2300, 23328,  9948, 22248, 13019,   329, 18271,  4037,
        9935], dtype=int64)

In [140]:
n = 51
print(np.take(tfidf.get_feature_names(), test_vec[n, :].toarray().argsort()[0][::-1])[:10])
print(np.take(tfidf_s.get_feature_names(), test_vec_s[n, :].toarray().argsort()[0][::-1])[:10])

data['LG생활건강']['X']['text'].iloc[cv_sets[0][1]].iloc[n]

['강보합권에서' '4월' '고용' '미국의' '하지만' '있다' '외국인과' '내리고' '동반' '매도에도']
['강보합권에서' '고용' '4월' '미국의' '감소했다고' '노동부는' '예상은' '조정치' '계절' '치솟았다']


'코스피가 외국인과 기관의 동반 매도에도 강보합권에서 출발하고 있다. 11일 오전 9시 3분 현재 코스피는 전 거래일 대비 8.52포인트(0.44%)오른 1954.34를 기록하고 있다.이날 7.69포인트(0.40%) 상승 출발한 지수는 장 초반 외국인과 기관이 동반 \'팔자\'로 나섰지만 개인 중심의 매수세가 유입되면서 1950선 강보합권에서 움직이고 있다. 지난주 코스피는 코로나19를 둘러싼 발병지 논란으로 미중간 무역분쟁 우려가 재부각되면서 매물이 출회했다. 하지만 미국 대형 기술주와 반도체 중심으로 강세를 보이면서 반등을 보인 데 힘입어 낙폭을 축소했다. 마감. 코스닥 지수는 기획재정부의 \'한국판 뉴딜\' 부양정책 추진 발표에 주 후반 강세를 보였다.앞서 지난 주말 뉴욕증시는 미국의 4월 고용지표가 우려보다는 덜 나빴던 데 힘입어 큰 폭으로 올랐다.미 노동부는 지난 4월 비농업부문 고용이 2050만 명(계절 조정치) 감소했다고 발표했다. 실업률은 3월의 4.4%에서 14.7%로 치솟았다. 고용 감소 규모와 실업률 모두 사상 최악 수준의 수치를 기록했다.하지만 시장이 우려했던 것보다는 나쁘지 않았던 점이 투자자들에게 안도감을 제공했다. 월스트리트저널이 집계한 시장 예상은 실업률이 16%로 치솟고, 고용은 2150만 명 감소하는 것이었다. 4월 실직자의 78% 이상이 \'일시해고\' 상태인 것으로 조사된 점도 향후 고용 반등에 대한 기대를 자극했다.서상영 키움증권 연구원은 "한국 증시는 글로벌 경제 재개에 대한 기대와 기업들의 급격한 이익 추정치 하향 조정에 따른 높은 밸류에이션, 미국의 코로나 19 확진자 추이 및 미중 무역협상 점검, 미국종양임상학회(ASCO) 초록 결과에 따른 제약, 바이오 업종의 변동성 확대 등을 감안 시 뚜렷한 방향성을 보이기 보다는 지난주에 이어 업종별, 종목별 차별화가 지속될 것으로 예상한다"고 말했다.이날 유가증권시장에서는 업종별로 의료정밀, 의약품, 운송장비, 서비스업, 종이목재 등이 오르고 있고 섬유의복, 건설업, 유통업,

# 전체 데이터 tf-idf & cosine-similiarity

In [218]:
x_all = pd.DataFrame()

for key in data.keys():
    x_tmp = data[key]['X']
    x_all = x_all.append(x_tmp)

In [219]:
len(x_all)

39016

In [144]:
train_period=60
test_period=29
tscv = TimeBasedCV(train_period=train_period, test_period=test_period, freq='days')
cv_sets = tscv.split(x_all.reset_index(drop=True), validation_split_date=None, date_column='timestamps', gap=0)

Train period: 2020-03-02 <= day < 2020-05-01 , Test period 2020-05-01 <= day < 2020-05-30 # train records 26608 , # test records 12408


In [145]:
tfidf = TfidfVectorizer(sublinear_tf=False)
tfidf.fit(x_all['text'].iloc[cv_sets[0][0]])

tfidf_s = TfidfVectorizer(sublinear_tf=True)
tfidf_s.fit(x_all['text'].iloc[cv_sets[0][0]])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [146]:
test_vec = tfidf.transform(x_all['text'].iloc[cv_sets[0][1]])
test_vec_s = tfidf_s.transform(x_all['text'].iloc[cv_sets[0][1]])

In [153]:
n = 450
print(np.take(tfidf.get_feature_names(), test_vec[n, :].toarray().argsort()[0][::-1])[:20])
print(np.take(tfidf_s.get_feature_names(), test_vec_s[n, :].toarray().argsort()[0][::-1])[:20])

x_all['text'].iloc[cv_sets[0][1]].iloc[n]

['재고관련' '배터리' '항공유와' 'sk이노베이션은' '관련해선' '휘발유' '1분기' '손실이' '목표를' '15만배럴'
 '중이라' '고밀도' '크랙' '손익에' '수송용' '있어선' '두바이' '원유가' '생산물량' '이노베이션을']
['재고관련' '항공유와' '관련해선' '휘발유' 'sk이노베이션은' '배터리' '15만배럴' '중이라' '수송용' '고밀도'
 '크랙' '손익에' '있어선' '두바이' '손실이' '목표를' '생산물량' '원유가' '이노베이션을' '감안한다면']


'[서울=뉴시스] 김지은 기자 = SK이노베이션은 6일 올해 배터리 매출 목표를 10% 내외 수준에서 하향 조정한다"고 밝혔다. SK이노베이션은 이날 1분기 실적 발표 이후 열린 콘퍼런스콜에서 "앞서 올해 매출 목표를 2조원으로 제시했는데 코로나19 확산으로 생산물량 조정 등이 불가피하게 됐다"고 말했다.다만 "손익에 있어선 수율 개선, 비용 최적화 등을 통해 당초 목표치를 유지하려 한다"고 했다.배터리사업은 유럽, 중국, 미국 등에서 생산시설을 확충 중이다. 예정대로 완공되면 2022년에는 연 60GW, 2023년에는 연 70GW의 생산능력을 보유하게 된다.전고체 배터리 개발과 관련해선 "글로벌 전문가와 함께 연구 중"이라며 "오픈 이노베이션을 채택해 고밀도 에너지 배터리 개발에 속도를 낼 것"이라고 부연했다.다만 "기술 발전 수준을 감안한다면 비즈니스 모델로 이어지기까지는 시간이 걸릴 것"이라고 언급했다.미국에서의 LG화학과의 소송과 관련해선 "구체적으로 말하는 것은 결과에 영향 미칠 수 있기 때문에 추가 코멘트를 할 순 없다"면서도 "다양한 가능성에 대해 모니터링, 준비 중이라 상황에 따라 맞게 대응할 것"이라고 말했다.정제공장은 추가적인 가동률 조정에 나선다.SK이노베이션은 "SK에너지는 코로나19에 따른 전 세계적 항공유와 휘발유 수요 급감과 크랙 하락 등을 고려해 울산Complex를 보수적으로 가동할 계획"이라며 "특히 2분기엔 넘버5 정기보수 계획이 있어 1분기에 비해 15만배럴 줄일 계획이다"고 말했다.이어 "감압잔사유 탈황설비(VRDS)의 경우 성공적으로 조기 가동했지만 코로나19로 예측했던 것보다 가동을 다소 줄였다"며 "최적 운영으로 4만배럴을 정상 가동하고 있다"고 했다.항공유와 휘발유 등 수송용 석유제품의 정제마진은 2분기까지 약세가 이어지고 6월부터 점진적으로 상승할 것으로 내다봤다.1분기 재고관련손실 규모는 총 1조1138억원으로 집계됐다. 사업별로 석유사업이 9418억원, 화학이 1393억원, 윤활유 327억원이다.회사 관계자는 "

In [156]:
cos_sim = test_vec * test_vec.T

In [165]:
cos_sim[0, :].toarray()

array([[1.        , 0.03283852, 0.03414869, ..., 0.01393147, 0.01487106,
        0.02629644]])

In [171]:
print(np.sort(cos_sim[0, :].toarray())[0][::-1])

[1.         0.17610834 0.15317275 ... 0.         0.         0.        ]


In [179]:
cos_sim[0, :].toarray()[0].argsort()[::-1]

array([   0, 1324, 5484, ..., 8423, 7069, 2438], dtype=int64)

In [212]:
n = 106

similars = np.take(x_all['text'].iloc[cv_sets[0][1]], cos_sim[n, :].toarray()[0].argsort()[::-1])

print(similars[0:7])

1602     지난 25일 김정태 하나금융그룹 회장과 조용병 신한금융그룹 회장이 글로벌 사업을 위...
1600     [헤럴드경제=금융팀] 지난 25일 김정태 하나금융그룹 회장과 조용병 신한금융그룹 회...
1000     (서울=뉴스1) 민정혜 기자,김도엽 기자 = "국내에선 두 금융그룹이 경쟁관계에 있...
16120    [아시아경제 이동우 기자] 우마르 하디(Umar Hadi) 주한 인도네시아 대사는 ...
15889    [아시아경제 이동우 기자] 우마르 하디(Umar Hadi) 주한 인도네시아 대사는 ...
982      국내 금융그룹 중 1위(순이익 기준)를 달리고 있는 와 3위 가 글로벌 경쟁력 강화...
1002     자산 기준 국내 1위 금융그룹인 와 3위인 가 국내 금융지주 최초로 글로벌 경쟁력 ...
Name: text, dtype: object


In [220]:
similars.iloc[0]

'지난 25일 김정태 하나금융그룹 회장과 조용병 신한금융그룹 회장이 글로벌 사업을 위해 손을 잡았다. 양해각서(MOU)는 보통 뭔가를 ‘같이 하자’는 뜻이지만, 이 날은 좀 달랐다. ‘다투지 말자’는 게 핵심이다. 이번 협약에는 ‘과당경쟁을 지양한다’는 대목이 들어갔다.이른바 ‘신남방’으로 불리는 동남아시아 금융시장에서는 그야말로 한국 금융사들이 ‘바글바글’하다. 금융감독원에 따르면 지난해 말 기준 국내은행들이 나라 밖에 낸 점포는 195곳이다. 이 가운데 69.2%(135곳)는 아시아권역이다. 베트남에만 우리나라 은행 점포가 19개가 몰렸다. 중국, 인도(16개), 미얀마(14개), 홍콩(11개), 캄보디아(10개) 등은 국내 은행들이 진출한 대표적인 지역이다.은행을 포함해 금융그룹 내 계열사로 따지면 밀도는 더 높아진다. 인도네시아, 베트남, 미얀마, 캄보디아에서는 우리 금융회사 끼리 경쟁을 벌일 정도다.인도네시아에는 신한·하나금융이 각각 43개, 72개 지점(은행, 캐피탈 등)을 거느리고 있다. 우리금융은 2014년 현지 소다라은행을 인수했고 현재 157개 영업점을 운영하고 있다.진출 초기엔 현지법인, 지점을 늘리던 금융사들의 글로벌 전략은 이제 현지 금융사 인수합병(M&A), 지분 매입 등으로 진화했다. 기존에는 현지에 진출한 국내 기업들을 주요 고객으로 삼았다면, 이제는 현지 고객들을 직접 겨냥한 리테일 서비스를 늘려가고 있다.비슷한 전략으로 경쟁하다보니 ‘출혈 경쟁’도 벌어진다.한 은행 관계자는 “지난해 몇몇 은행이 인도네시아, 캄보디아의 현지 금융사를 인수하는 과정에서 주가 순자산비율(PBR)의 3배 가까이 지불한 사례가 있다”며 “지나치다는 말들도 있었다”고 전했다.지난해 하나금융그룹은 베트남투자개발은행(BIDV)의 전략투자자(SI)로 지분 15%를 사는데 1조148억원을 지불했다. 덕분에 BIDV 자본은 전년 말 2조7245억원대에서 3조8826억원(100동=5원 )대로 급증했다. 순익과 배당을 고려할 때 지분 15%를 확보하면서 자본을 40%

In [221]:
similars.iloc[3]

'[아시아경제 이동우 기자] 우마르 하디(Umar Hadi) 주한 인도네시아 대사는 현지에 진출한 국내 기업인들과 만나 신종 코로나바이러스감염증(코로나19) 사태에 따른 양국 경제협력 방안을 모색할 예정이다.22일 재계에 따르면 하디 대사는 오는 27일 여의도 전경련회관에서 우리 기업인들과 만나 양국 교역 확대를 위한 적극적인 현지 투자 방안을 요청할 계획이다. 하디 대사는 포스트코로나 시대에 맞춰 자국 내 투자 강화를 위한 기업 친화적인 정책을 소개하고, 관광 산업 등 분야에서 양국 기업 간 긴밀한 협력 관계를 추진하자는 뜻을 전달할 것으로 알려졌다.대한무역투자진흥공사에 따르면 인도네시아는 2018년 역대 최대 규모인 85억7000만달러(9조6000억원)의 무역적자를 기록한 후 현재까지 회복이 더딘 상황이다.권태신 전경련 부회장은 이 자리에서 하디 대사에게 현지 진출한 국내 기업의 코로나19 피해를 적극적으로 지원해줄 것을 요청할 계획이다. 또 현지 진출한 20여개의 국내 주요 기업인들이 참석해 경영 애로사항을 건의할 방침이다. 국내 대표적인 진출 기업으로는 1969년 코린도그룹을 시작으로 현재 삼성전자, LG전자, CJ, 롯데마트, SK에너지 등이 있다.재계 관계자는 "최근 국내 주요 기업이 인도네시아로 생산라인을 이전하는 등 현지 정부가 우리 기업의 투자 유치를 위해 적극적으로 노력하고 있다"며 "이와 관련한 인도네시아 정부의 구체적인 사항이 나오길 기대하고 있다"고 말했다. 이동우 기자 dwlee@asiae.co.kr <ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제 무단전재 배포금지>'